# Load NREL data files to mysql with DATETIME column.

This notebook loads previously altered CSV files containing NREL Typical Meteorological Year data sets in a MySQL database.  

The files must have been processed using __format_nrel_data.ipynb__ (found in the repository) to convert the hour data from 01-24 to 00-23.  This operation adds an extra column, Time_fmt, to the raw data.  See the notebook for a detailed explanation.

Get the MySQL connector package.

In [97]:
import mysql.connector

Open the connection to the database.

In [98]:
cnx = mysql.connector.connect(user='python_connection',
                              password='demo_pass', 
                              database='NREL_TMY')

cursor = cnx.cursor()

Look up the list of properly formatted data sets available

In [99]:
!ls ./TMY_data/ | grep 'fmt'

alturas_CA_fmt.CSV
denver_CO_fmt.CSV
ely_MN_fmt.CSV
fort_lauderdale_FL_fmt.CSV
houston_TX_fmt.CSV
knoxville_TN_fmt.CSV
lafayette_IN_fmt.CSV
laramie_WY_fmt.CSV
new_york_NY_fmt.CSV
portland_OR_fmt.CSV
san_diego_CA_fmt.CSV
seattle_WA_fmt.CSV


Define the 'city_ST' string for the file and table names.

In [100]:
city ='ely_MN'

Define a string with the SQL syntax for creating a table, then execute the string in the cursor.

In [101]:
make_table_str = ("""CREATE TABLE """ + city +
"""\n(
Date                   varchar(30),
Time                   varchar(30),
Time_fmt               varchar(30),
ETR                    float(14,4),
ETRN                   float(14,4),
GHI                    float(14,4),
GHI_source             varchar(30),
GHI_uncert             tinyint,
DNI                    float(14,4),
DNI_source             varchar(30),
DNI_uncert             tinyint,
DHI                    float(14,4),
DHI_source             varchar(30),
DHI_uncert             tinyint,
GH_illum               float(14,4),
GH_illum_source        varchar(30),
Global_illum_uncert    tinyint,
DN_illum               float(14,4),
DN_illum_source        varchar(30),
DN_illum_uncert        tinyint,
DH_illum               float(14,4),
DH_illum_source        varchar(30),
DH_illum_uncert        tinyint,
Zenith_lum             float(14,4),
Zenith_lum_source      varchar(30),
Zenith_lum_uncert      tinyint,
TotCld                 float(14,4),
TotCld_source          varchar(30),
TotCld_uncert          tinyint,
OpqCld                 float(14,4),
OpqCld_source          varchar(30),
OpqCld_uncert          tinyint,
Dry_bulb               float(14,4),
Dry_bulb_source        varchar(30),
Dry_bulb_uncert        tinyint,
Dew_point              float(14,4),
Dew_point_source       varchar(30),
Dew_point_uncert       tinyint,
RHum                   float(14,4),
RHum_source            varchar(30),
RHum_uncert            tinyint,
Pressure               float(14,4),
Pressure_source        varchar(30),
Pressure_uncert        tinyint,
Wdir                   float(14,4),
Wdir_source            varchar(30),
Wdir_uncert            tinyint,
Wspd                   float(14,4),
Wspd_source            varchar(30),
Wspd_uncert            tinyint,
Hvis                   float(14,4),
Hvis_source            varchar(30),
Hvis_uncert            tinyint,
CeilHgt                float(14,4),
CeilHgt_source         varchar(30),
CeilHgt_uncert         tinyint,
Pwat                   float(14,4),
Pwat_source            varchar(30),
Pwat_uncert            tinyint,
AOD                    float(14,4),
AOD_source             varchar(30),
AOD_uncert             tinyint,
Alb                    float(14,4),
Alb_source             varchar(30),
Alb_uncert             tinyint,
Lprecip_depth          float(14,4),
Lprecip_quantity       float(14,4),
Lprecip_source         float(14,4),
Lprecip_uncert         varchar(30),
PresWth                varchar(30),
PresWth_source         varchar(30),
PresWth_uncert         varchar(30)
);""")

#print(make_table_str)

# Execute
cursor.execute(make_table_str)

Define string with LOAD DATA commands for CSV file.  After the execution the changes must be committed.

In [102]:
load_data_str = (r"""LOAD DATA LOCAL INFILE './TMY_data/"""+ city +r"""_fmt.CSV'
INTO TABLE """+ city +r"""
COLUMNS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
ESCAPED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;""")


#print(load_data_str)
cursor.execute(load_data_str)

# Commit the changes!
cnx.commit()

Add a new column of type DATETIME and populate it using the formatted time, Time_fmt.  This facilitates writing queries on the table. 

In [103]:
new_col_str = """ALTER TABLE """+ city +r""" ADD Date_time DATETIME NOT NULL DEFAULT '2000-01-01 14:30:00' AFTER Time_fmt;"""
cursor.execute(new_col_str)
cnx.commit()
#print(new_col_str)

set_str = r"""SET sql_mode='';"""
cursor.execute(set_str)
cnx.commit()
#print(set_str)

update_str = """UPDATE """+ city +r""" SET Date_time = STR_TO_DATE(Time_fmt, '%Y-%m-%d-%H:%i:%s');"""
cursor.execute(update_str)
cnx.commit()
#print(update_str)

In [104]:
cnx.close()